In [1]:
class ClassifierMultiDimen():
    
    def __init__(self):
        self.frequency = {}
        self.accuracy = -1
        self.prf = []
        self.conf_matrix = []
        
        
    def classify(self, x_train, y_train, x_test, y_test): 
#         print 'Len of x_train', len(x_train)
#         print 'Len of y_train', len(y_train)
#         print 'Len of x_test', len(x_test)
#         print 'Len of y_test', len(y_test)
        
        if len(x_train) != len(y_train) or len(x_test) != len(y_test):
            raise ValueError("Length mismatch of data and labels") 
        
        ### compute frequency of labels in train data ###
        self.frequency = dict([(i,0) for i in np.unique(y_train)])
        for number in y_train:
            self.frequency[number]+=1 
        for key in self.frequency:
            self.frequency[key] = float(self.frequency[key]) / len(y_train)

            
        print "Number of samples:", len(x_train)
        print "Number of features:", len(x_train[0])
        
        ### setting up classifier ###
        clf = RandomForestClassifier(max_depth=10, random_state=0)
        clf.fit(x_train, y_train) 

        ### compute model performance for test data###
        y_true = y_test
        y_pred = clf.predict(x_test)
        self.accuracy = sklearn.metrics.accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)
        self.prf = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred, beta=1.0, pos_label=1, average='weighted', sample_weight=None)
        self.conf_matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
        print "Performance computed: ", self.conf_matrix
        
        
    
#     # x_data should have only columns for data
#     def classify_kfold(self, x_data, y_data, encode):
#         k_frequency = []
#         k_accuracy = []
#         k_prf = []
#         k_conf_matrix = []
#         for i in range(k):
#             print("Iteration ", i)
            
#             # setting up train & test data
#             train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(
#                 x_data, y_data, test_size=0.2, random_state=i)
        
#             # return is there not enough unique labels
#             if len(set(train_labels)) != len(set(test_labels)):
#                 print 'Not enough samples for this iteration'
#                 continue
                
#             self.classify(train_data, train_labels, test_data, test_labels, encode)
#             k_frequency.append(self.frequency)
#             k_accuracy.append(self.accuracy)
#             k_prf.append(self.prf)
#             k_conf_matrix.append(self.conf_matrix)
        
#         self.frequency = k_frequency
#         self.accuracy = k_accuracy
#         self.prf = k_prf
#         self.conf_matrix = k_conf_matrix
        
        
       
    # x_data should have two columns for user and data
    def classify_leave_one_out(self, x_data_list, y_data, samples_req, users, encode=True):
        # get list of users
        self.users = users
        
        k_frequency = []
        k_accuracy = []
        k_prf = []
        k_conf_matrix = []
        
        
        
        for i in range(len(self.users)):
        #for i in [1,2,5]:
            current_user = self.users[i]
            print "Iteration ", i, current_user
            
            # list for merging all data representations
            all_train_data, all_test_data, all_train_labels, all_test_labels = [], [], [], []
            
            # setting up train & test data from each dataset
            for i in range(len(x_data_list)): 
                x_data = x_data_list[i]
                
                # indices for current user
                cu_indices = [i  for i in range(len(x_data)) if x_data[i][0] == current_user]

                # N pos & neg index for this user [to be added for training]
                cu_pos_index, cu_neg_index = [],[]
                for cu_index in cu_indices:
                    if y_data[cu_index] == 1 and len(cu_pos_index) < samples_req:
                        cu_pos_index.append(cu_index)
                    if y_data[cu_index] == -1 and len(cu_neg_index) < samples_req:
                        cu_neg_index.append(cu_index)
                    if len(cu_pos_index) == samples_req and len(cu_neg_index) == samples_req:
                        break    
                cu_indices = [i for i in cu_indices if i not in cu_pos_index and i not in cu_neg_index]

                if len(cu_indices) < 10: # or len(cu_pos_index) < samples_req or len(cu_neg_index) < samples_req:
                    print 'Not enough samples for this iteration'
                    print 'Total-Pos-Neg labels', len(cu_indices) #, len(cu_pos_index), len(cu_neg_index)
                    break 


                train_data = [x_data[i][1]  for i in range(len(x_data)) if i not in cu_indices]
                test_data = [x_data[i][1]  for i in range(len(x_data)) if i in cu_indices]
                train_labels = [y_data[i]  for i in range(len(y_data)) if i not in cu_indices]
                test_labels = [y_data[i]  for i in range(len(y_data)) if i in cu_indices]

                # return is there not enough unique labels
                if len(set(train_labels)) != len(set(test_labels)):
                    print 'Not enough samples for this iteration'
                    print 'Train-test labels', len(set(train_labels)), len(set(test_labels))
                    break
                
                        
                ### encode data  ###
                if encode == True:
                    # train AE (only on train data)
                    input_layer_size = len(train_data[0])
                    ae = AETrainer(input_layer_size)
                    ae.set_data(train_data, train_labels)
                    ae.train_model()
                    encoded_data = ae.encode_data(train_data)
                    train_data = encoded_data.data.numpy().tolist()
                    encoded_data = ae.encode_data(test_data)
                    test_data = encoded_data.data.numpy().tolist()
                    
                    
                
                # merge data
                for j in range(len(train_data)):
                    if len(all_train_data) == j:
                        all_train_data.append(train_data[j])
                    else:
                        for td in train_data[j]:
                            all_train_data[j].append(td)
                for j in range(len(test_data)):
                    if len(all_test_data) == j:
                        all_test_data.append(test_data[j])
                    else:
                        for td in test_data[j]:
                            all_test_data[j].append(td)
                if len(all_train_labels) == 0:
                        all_train_labels = train_labels
                if len(all_test_labels) == 0:
                        all_test_labels = test_labels
            
            # go to next user if no data is computed for this user
            if len(all_train_data) == 0 or len(all_test_data) == 0:
                continue
            
            self.classify(all_train_data, all_train_labels, all_test_data, all_test_labels)
            k_frequency.append(self.frequency)
            k_accuracy.append(self.accuracy)
            k_prf.append(self.prf)
            k_conf_matrix.append(self.conf_matrix)

            self.frequency = k_frequency
            self.accuracy = k_accuracy
            self.prf = k_prf
            self.conf_matrix = k_conf_matrix 


In [ ]:
class ClassifierMultiDimenHelper:

    @staticmethod
    def compute_svm_accuracy(xy_data_list, is_leave_one_user, samples_req=3, users=[]):
        # setting up the data
        y_data = [d[4] for d in xy_data_list[0]]
        x_data = []
        for xy_data in xy_data_list:
            if is_leave_one_user == True:
                x_data.append([[d[0], d[5]] for d in xy_data])
            else:
                x_data.append([d[5] for d in xy_data])

        # classify
        model = ClassifierMultiDimen()
        if is_leave_one_user == True:
            model.classify_leave_one_out(x_data, y_data, samples_req, users)
        else:
            model.classify_kfold(x_data, y_data)

        return model